<a href="https://colab.research.google.com/github/sanfordmalman/My-publications/blob/master/BookNLP_RecEnvGov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Demo for running BookNLP on an input text.  To use a GPU in Google Colab, change the following:

`Runtime > Change runtime type > Hardware accelerator > GPU`

To execute this notebook, sign in to Google and then run all cells

`Runtime > Run all`

In [ ]:
!pip install booknlp

In [ ]:
!wget https://www.gutenberg.org/files/208/208.txt -O 208_daisy_miller.txt

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
from booknlp.booknlp import BookNLP

In [ ]:
model_params={
		"pipeline":"entity,quote,supersense,event,coref", 
		"model":"big", 
	}

booknlp=BookNLP("en", model_params)

In [ ]:
inputFile="208_daisy_miller.txt"
outputDir="208_daisy_miller/"
idd="208_daisy_miller"

booknlp.process(inputFile, outputDir, idd)

In [ ]:
import json
from collections import Counter

In [ ]:
def proc(filename):
    with open(filename) as file:
        data=json.load(file)
    return data

In [ ]:
data=proc("208_daisy_miller/208_daisy_miller.book")

In [ ]:
def get_counter_from_dependency_list(dep_list):
    counter=Counter()
    for token in dep_list:
        term=token["w"]
        tokenGlobalIndex=token["i"]
        counter[term]+=1
    return counter

In [ ]:
for character in data["characters"]:
    
    agentList=character["agent"]
    patientList=character["patient"]
    possList=character["poss"]
    modList=character["mod"]
    
    character_id=character["id"]
    count=character["count"]

    referential_gender_distribution=referential_gender_prediction="unknown"

    if character["g"] is not None and character["g"] != "unknown":
        referential_gender_distribution=character["g"]["inference"]
        referential_gender=character["g"]["argmax"]

    mentions=character["mentions"]
    proper_mentions=mentions["proper"]
    max_proper_mention=""

    # just print out information about named characters
    if len(mentions["proper"]) > 0:
        max_proper_mention=mentions["proper"][0]["n"]
        
        print(character_id, count, max_proper_mention, referential_gender)

        print()
        printTop=10
        for k, v in get_counter_from_dependency_list(possList).most_common(printTop):
            print("\tposs\t%s %s" % (v,k))
        print()
        for k, v in get_counter_from_dependency_list(agentList).most_common(printTop):
            print("\tagent\t%s %s" % (v,k))        
        print()
        for k, v in get_counter_from_dependency_list(patientList).most_common(printTop):
            print("\tpatient\t%s %s" % (v,k))       
        print()
        for k, v in get_counter_from_dependency_list(modList).most_common(printTop):
            print("\tmod\t%s %s" % (v,k))    
        print()